In [1]:
import xarray
import numpy as np
import matplotlib as mpl

import matplotlib.pyplot as plt  # plotting library

# Issue: Maybe downgrade shapely pip install shapely ==1.8.5
import cartopy.crs as ccrs  # Projections list
import cartopy.feature as cfeature # for coastlines

import plotly.graph_objects as go

# Replicate NASA colorscales

https://svs.gsfc.nasa.gov/31158/

In [109]:
cvals  = [0., 0.15, 0.3, 0.45, 0.5, 1.]
colors = ["#371229", "#B6222B", "#FF8E35", "#FCDC79", "white", "#62C3DB"]

norm = plt.Normalize(np.min(cvals), np.max(cvals))
tuples = list(zip(map(norm, cvals), colors))
nasa_cmap = mpl.colors.LinearSegmentedColormap.from_list("", tuples)

# Heatmap requres a colorscale with a list of tuples
nasa_cscale = [(0,"#371229"),(0.15,"#B6222B"),(0.3,"#FF8E35"),(0.45,"#FCDC79"),(0.5,"white"), (1,"#62C3DB")]

In [55]:
# Load the data
gmb = xarray.open_dataset(
    "/home/kim/new_massbalance/mass_balance_ConvNP/data/GRACE_GFZ/GRAVIS-3_2002095-2023334_COSTG_0100_AIS_GRID_TUD_0003.nc")

# Convert to kg my multiplying by m^2, convert to gt by dividing by 1e12
# Set new item in xarray dataset
# coordinates are swopped this way
gmb["dm_gt"] = ((gmb.area * gmb.dm)/1e12).transpose("time", "y", "x")

In [84]:
# diff has one dimension less so we expand
empty = np.expand_dims(np.empty(gmb.dm_gt.shape[1:3]), axis = 0)
empty[:] = np.nan

gmb["dm_gt_monthly"] = (dims := ("time", "y", "x") , data := np.concatenate((empty, gmb.dm_gt.diff(dim = "time")), axis = 0))

# Mass change in [Gt/month]

In [ ]:
t = 150

# Needs to be a dataset for this vis to work
fig = go.Figure(go.Heatmap(
    z = gmb.dm_gt_monthly[t].values,
    # colorscale = 'RdBu',
    colorscale = nasa_cscale,
    zmid = 0,))

fig.update_layout(
    title = str("Antarcic mass change in month ") + str(gmb.dm_gt[t].time.values)[0:10],
)

fig.update_traces(colorbar_orientation = "v")
fig.update_traces(colorbar_title = "Gt")
fig.update_traces(colorbar_len = 0.7)

fig.update_layout(
    height = 800,
    width = 800,
    yaxis_scaleanchor = "x",
)

fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.update_layout(template = "plotly_white")
fig.update_layout(font_family = "Lato")

fig.show()

# Mass balance over time

longer than a month

In [ ]:
# 0 value is nan so start at 1
t_start = 1
t_end = 200

# Needs to be a dataset for this vis to work
fig = go.Figure(go.Heatmap(
    z = gmb.dm_gt_monthly[t_start : t_end].values.sum(axis = 0),
    # colorscale = 'RdBu',
    colorscale = nasa_cscale,
    zmid = 0,))

fig.update_layout(
    title = str("Antarcic mass balance from ") + str(gmb.dm_gt_monthly[t_start].time.values)[0:10] + str(" to ") + str(gmb.dm_gt_monthly[t_end].time.values)[0:10],
)

fig.update_traces(colorbar_orientation = "v")
fig.update_traces(colorbar_title = "Gt")
fig.update_traces(colorbar_len = 0.7)

fig.update_layout(
    height = 800,
    width = 800,
    yaxis_scaleanchor = "x",
)

fig.update_xaxes(visible = False)
fig.update_yaxes(visible = False)

fig.update_layout(template = "plotly_white")
fig.update_layout(font_family = "Lato")

fig.show()

# Slider

In [115]:
# range = - np.nanmin(gmb.dm_gt_monthly) + np.nanmax(gmb.dm_gt_monthly)

# fix the colorscale to max cum.
min = np.nanmin(gmb.dm_gt_monthly.sum(axis = 0))
# max = np.nanmax(gmb.dm_gt_monthly.sum(axis = 0))

In [ ]:
fig = go.Figure()

# add a trace for each time step
for i in np.arange(1, len(gmb.dm.time)):
    fig.add_trace(go.Heatmap(
        z = gmb.dm_gt_monthly[1:i].values.sum(axis = 0),
        colorscale = nasa_cscale,
        zmin = min,
        zmax = - min, # max or -min
        zmid = 0,
    ))

    fig.update_traces(colorbar_orientation = "v")
    fig.update_traces(colorbar_title = "Gt")
    fig.update_traces(colorbar_len = 0.7)

    fig.update_layout(
        height = 800,
        width = 800,
        yaxis_scaleanchor = "x",
    )

    fig.update_xaxes(visible = False)
    fig.update_yaxes(visible = False)

    fig.update_layout(template = "plotly_white")
    fig.update_layout(font_family = "Lato")

# Make 0th trace visible
fig.data[0].visible = False

# Create and add slider
steps = []
for i in np.arange(len(fig.data)):
    step = dict(
        method = "update",
        args = [{"visible": [False] * len(fig.data)},
              {"title": "AIS mass balance in month: " + str(gmb.dm[i].time.values)[0:10]}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 0,
    currentvalue = {"prefix": "Month: "},
    # pad = {"t": 10},
    steps = steps
)]

fig.update_layout(
    sliders = sliders
)

for i in np.arange(len(fig.data)):
    fig['layout']['sliders'][0]['steps'][i]['label'] = str(i)

fig.show()